In [2]:
import pandas as pd
from cleantext import clean
import re
import torch.nn.functional as F


import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time
import math

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [6]:
dataset_path = "../datasets/"

In [3]:
window_size = 310
slide = 70

In [4]:
def make_windows(content, w_size, slide):
    if slide > w_size :
        raise Exception()
    tokens = content.split(" ")
    l = len(tokens)    
       
    n = math.ceil(l / (w_size-slide))        

    if n==1 :
        return [content]
    else :
        windows = []
        for i in range(n) :            
            x = i*slide
            if x < l :
                y = x + w_size
                w = tokens[x:y]
                windows.append(" ".join(w))
        
        return windows

In [7]:
fiches = pd.read_csv(dataset_path+"xp-documents-fiches (2).csv")

fiches["cleaned_text"] = fiches["text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
fiches["tokens"] = fiches["cleaned_text"].apply(lambda t : len(str(t).split(" ")))

fiches = fiches[fiches["tokens"]<3000]
fiches["windows"] = fiches["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
fiches.drop(columns=["cleaned_text", "text"], inplace=True)
fiches = fiches.explode("windows", ignore_index=True)
fiches.rename(columns={"windows": "text"}, inplace=True)
fiches["cdtnId-fiches"] = fiches["cdtnId"]
fiches["cdtnId"] = fiches["cdtnId"].apply(lambda x : x.split("-")[0])

In [8]:
contribs = pd.read_csv(dataset_path + "xp-documents-contribs (1).csv")

In [9]:
contribs.drop_duplicates(subset=["answers.generic.text"], inplace=True)
contribs["cleaned_text"] = contribs["answers.generic.text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
contribs["tokens"] = contribs["cleaned_text"].apply(lambda t : len(str(t).split(" ")))
contribs["windows"] = contribs["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
contribs.drop(columns=["cleaned_text","answers.generic.text","text"], inplace=True)
contribs = contribs.explode("windows", ignore_index=True)
contribs.rename(columns={"windows": "text"}, inplace=True)

In [10]:
contribs["slug"] = contribs["slug"].apply(lambda s: "-".join(s.split("-")[1:]))

In [11]:
contribs_tensor = torch.load(dataset_path + "contribs_tensor3.pt")
fiches_tensor = torch.load(dataset_path + "fiches_tensor3.pt")
tensor_docs = torch.vstack([fiches_tensor, contribs_tensor])

In [12]:
passages_set = pd.concat([fiches, contribs]).reset_index()
passages_set.shape

(4795, 9)

In [13]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

In [14]:
# fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie	
doc1 = "6e888b3f6e"

# contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences
doc2 = "1100b865de"

passages_set[passages_set["cdtnId"] == doc1]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
3568,3568,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,que se passe-t-il si vous tombez malade pendan...,6e888b3f6e
3569,3569,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,de congés restants si la convention collective...,6e888b3f6e
3570,3570,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,congés si vous êtes malade (maladie non-profes...,6e888b3f6e
3571,3571,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,arret-maladie-et-conges-payes-annuels,-,741,prise de congés en cours dans l'entreprise ou ...,6e888b3f6e


In [15]:
passages_set[passages_set["cdtnId"] == doc2]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
4751,23,contributions,1100b865de,"Si le salarié est malade pendant ses congés, q...",si-le-salarie-est-malade-pendant-ses-conges-qu...,NaN,261,le salarié est malade avant son départ en cong...,NaN
4752,24,contributions,1100b865de,"Si le salarié est malade pendant ses congés, q...",si-le-salarie-est-malade-pendant-ses-conges-qu...,NaN,261,d'un arrêt maladie. le salarié perçoit une ind...,NaN


In [16]:
doc_passages = passages_set[passages_set["cdtnId"] == search_id]
indices = doc_passages.index
tensor_docs[indices]
retrieval_scores = (tensor_docs[indices] @ tensor_docs.T) * 100
retrieval_scores.size()
topk = torch.topk(retrieval_scores, 100)


NameError: name 'search_id' is not defined

In [274]:
topk.indices.size()

torch.Size([3, 100])

In [275]:
top_indices = topk.indices.tolist()

In [282]:
passages_set.loc[top_indices[0][:10]].text.iloc[doc_passages.shape[0]]

"l'employeur apporte la justification de la perturbation du fonctionnement de l'entreprise . il justifie également de la nécessité de remplacer définitivement le salarié absent par un cdi . indemnités si le salarié en remplit les conditions, il a droit aux indemnités suivantes : indemnité légale ou conventionnelle de licenciement indemnité compensatrice de congés payés indemnité compensatrice de préavis à noter des dispositions conventionnelles peuvent prévoir des conditions d'indemnisation plus favorables. motif disciplinaire conditions le salarié peut être licencié s'il a commis une faute . elle peut être la conséquence de l'une des situations suivantes : non-respect par le salarié malade de ses obligations (par exemple, l'envoi tardif d'un arrêt de travail malgré les mises en demeure de justifier de l'absence) faute commise avant l'arrêt maladie procédure disciplinaire engagée avant l'arrêt maladie procédure l'employeur doit respecter la procédure de licenciement pour motif personne

In [281]:
doc_passages.text.iloc[0]

"la clause de non-concurrence est une clause insérée dans le contrat de travail. elle vise à limiter la liberté d'un salarié d'exercer, après la rupture de son contrat, des fonctions équivalentes chez un concurrent ou à son propre compte. pour être valable, la clause doit respecter certains critères. critères de validité la clause de non-concurrence n'est pas définie par la loi , mais a été précisée par des décisions de justice, c'est-à-dire par l a jurisprudence . pour qu'elle soit applicable, la clause de non-concurrence doit répondre à certains critères cumulatifs définis qui conditionnent sa validité. la clause doit être écrite dans le contrat de travail (ou prévue dans la convention collective ). la clause est applicable uniquement si elle veille à protéger les intérêts de l'entreprise (quand le salarié est en contact direct avec la clientèle par exemple). elle ne doit pas empêcher le salarié de trouver un emploi ailleurs. la clause de non-concurrence s'applique : dans le temps (s

In [25]:
def find_similar(cdtnId):
    k = 20

    doc_passages = passages_set[passages_set["cdtnId"] == search_id]
    indices = doc_passages.index
    tensor_docs[indices]
    retrieval_scores = (tensor_docs[indices] @ tensor_docs.T) * 100
    retrieval_scores.size()
    topk = torch.topk(retrieval_scores, k)
    
    matches = topk.values.reshape(-1,).tolist()
    matching_indices = topk.indices.reshape(-1,).tolist()

    rows = passages_set.loc[matching_indices].copy()
    rows["similarity"] = matches
    results = rows[rows["cdtnId"] != cdtnId]

    return results




In [286]:
indices = passages_set[passages_set["cdtnId"] == doc1].index
indices
indices.to_list()

[3568, 3569, 3570, 3571]

In [30]:

search_id = '4c949978ea'


print(passages_set[passages_set["cdtnId"] == search_id].iloc[0][["source", "slug"]].to_list())
print(passages_set[passages_set["cdtnId"] == search_id].shape[0])
#print(passages_set[passages_set["cdtnId"] == search_id]["text"].to_list())

#print(find_similar(search_id)[["index","source","cdtnId","slug","similarity"]].to_string())
search_id


res = find_similar(search_id)
gb = res.groupby(["cdtnId", "slug", "source"])

#print( gb["similarity"].max())

scores = gb["similarity"].max() * (1+gb["similarity"].count() / 10)
scores.sort_values(ascending=False)[:10]

#grouped.mean(["similarity"]).sort_values(by=["similarity"], ascending=False)[:20]
#grouped.size()

# counts = gb.size().to_frame(name='counts')
# (counts
#     .join(gb.agg({'col3': 'mean'}).rename(columns={'col3': 'col3_mean'}))
#     .join(gb.agg({'col4': 'median'}).rename(columns={'col4': 'col4_median'}))
#     .join(gb.agg({'col4': 'min'}).rename(columns={'col4': 'col4_min'}))
#     .reset_index())
   



#.sort_values(by=["similarity"], ascending=False)[:20]

['fiches_service_public', 'conge-ou-temps-partiel-pour-creation-ou-reprise-dentreprise']
9


cdtnId      slug                                                                                                                    source                  
bb9cc8c27e  conge-sabbatique-du-salarie-dans-le-secteur-prive                                                                       fiches_service_public       382.955113
e4e05bc3be  le-conge-ou-temps-partiel-pour-creation-ou-reprise-dune-jeune-entreprise-innovante#Le-depart-en-conge-quelle-procedure  fiches_ministere_travail    290.704857
f3b51b5190  le-conge-ou-temps-partiel-pour-creation-ou-reprise-dune-jeune-entreprise-innovante#Le-depart-en-conge-quelle-procedure  fiches_ministere_travail    271.324533
            le-conge-ou-temps-partiel-pour-creation-ou-reprise-dune-jeune-entreprise-innovante#Quels-sont-les-avantages             fiches_ministere_travail    204.613303
e4e05bc3be  le-conge-ou-temps-partiel-pour-creation-ou-reprise-dune-jeune-entreprise-innovante#Quels-sont-les-avantages             fiches_ministere_travail   

In [241]:
def lookup(slug):
    slug_search = passages_set[passages_set["slug"] == slug]
    dd = slug_search["cdtnId"].drop_duplicates()
    n = dd.shape[0]
    if n == 1 :
        return dd.iloc[0]
    elif n == 0 :
        raise Exception("not found")
    else :
        raise Exception("ambiguous slug search")

In [254]:
lookup("quest-ce-quune-clause-de-non-concurrence")

'c5358a2fe7'

In [78]:
passages_set[passages_set["cdtnId"] == "c6c206dc8b"]

,index,source,cdtnId,title,slug,answers.generic.text,tokens,text,cdtnId-fiches
4274,4274,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement é...,le-contrat-de-travail-est-il-obligatoirement-e...,-,249,les contrats de travail suivants doivent oblig...,c6c206dc8b
4275,4275,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement é...,le-contrat-de-travail-est-il-obligatoirement-e...,-,249,le salarié est présumé être en cdi à temps ple...,c6c206dc8b
